In [1]:
import pandas as pd
import sys
sys.path.append("../scripts/")
from extractor import DataExtractor

In [2]:
data_extractor=DataExtractor()

In [3]:
with open('../data/20181024_d1_0830_0900.csv','r') as f:
    lines=f.readlines()

columns=lines[0].replace('\n','').split(';')
data=lines[1:]

In [4]:
columns[:4]

['track_id', ' type', ' traveled_d', ' avg_speed']

In [5]:
columns[4:]

[' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time']